<h1 align="center"><a href="https://github.com/sborquez/her2bdl"> Her2BDL</a> - Her2 Bayesian Deep Learning</h1>


<br>
<center>
<img src="images/utfsm.png" width="50%", align="center"/>
</center>
<h2 align="center">Evaluation</h2>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - utfsm - October 2020.</i>
</center>


# Setup Notebook

The first step is to sync this notebook with Google Drive, and change directory to gerumo repository.

In [ ]:
#!export HER2BDL_HOME="/home/sborquez/projects/her2bdl"
#!export HER2BDL_DATASETS="/home/sborquez/projects/her2bdl/datasets"
#!export HER2BDL_EXPERIMENTS="/home/sborquez/projects/her2bdl/experiments"

In [ ]:
## (Option A) Colab Setup
# Connect to your `Google Drive` and install some dependencies.
try:
    raise NotImplementedError
    from google.colab import drive
    colab_ = True
    drive.mount('/content/drive')
    #!pip install git+https://github.com/sborquez/her2bdl\
## (Option B) Local Setup
# Run locally.
except:
    colab_ = False
    
# Adhoc modules
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_DISABLE_PTX_JIT'] = "1"
os.environ["WANDB_MODE"] = 'dryrun'

from extra import *

# Her2BDL package
#if is not installed run these two line
HER2BDL_HOME=os.environ.get("HER2BDL_HOME")
%cd $HER2BDL_HOME

from her2bdl import *
%matplotlib inline

In [ ]:
config_folder = Path(os.environ.get("HER2BDL_EXPERIMENTS")) / "config"

# Configuration

In [ ]:
# Binary

## Aleatoric
#config_file = config_folder / "aleatoric_evaluation/efficientnet_b0_binary_classifier.yaml"

## Classification & Epistemic
#config_file = config_folder / "binary_classification_evaluation/efficientnet_b0_binary_classifier.yaml"

# HER2

## Aleatoric
#config_file = config_folder / "aleatoric_evaluation/hdxconv_c876.yaml"
#config_file = config_folder / "aleatoric_evaluation/hedconv_c876.yaml"
#config_file = config_folder / "aleatoric_evaluation/rgbconv_c876.yaml"

## Classification & Epistemic
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08.yaml"

## Aggregation & Epistemic

#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_base.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_base.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_base.yaml"

#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_threshold_Hh.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_threshold_Hl.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_threshold_Ih.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_threshold_Il.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_mixture_H.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d02_mixture_I.yaml"

#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_threshold_Hl.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_threshold_Hh.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_threshold_Il.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_threshold_Ih.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_mixture_H.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d05_mixture_I.yaml"

#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_threshold_Hl.yaml"
config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_threshold_Hh.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_threshold_Ih.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_threshold_Il.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_mixture_H.yaml"
#config_file = config_folder / "baseline_evaluation_warwick/hdxconv_c876_d08_mixture_I.yaml"

# Model Evaluation

Load and predict with your trained models. Evaluate and plot metrics.

In [ ]:
config = load_config_file(config_file)
run_dir = setup_experiment(config, mode="evaluation")

# Experiment paths and indentifiers
experiments_folder = config["experiment"]["experiments_folder"]
experiment_name    = config["experiment"]["name"]
experiment_folder  = Path(experiments_folder) / experiment_name
run_id             = config["experiment"]["run_id"]

# Evaluation parameters
batch_size  = config["evaluation"]["batch_size"]
evaluate_classification = config["evaluation"]["evaluate_classification"]
evaluate_aleatoric_uncertainty = config["evaluation"]["evaluate_aleatoric_uncertainty"]
evaluate_aggregation = config["evaluation"]["evaluate_aggregation"]
# Dataset
data_configuration = config["data"]
test_, input_shape, num_classes, labels = setup_generators(
    batch_size=batch_size, test_dataset=True,  **data_configuration
)
(test_dataset, steps_per_epoch) = test_
# Model architecture
model_configuration = config["model"]
model = setup_model(input_shape, num_classes, **model_configuration)

results=None

In [ ]:
# Evaluation Logguer
logger_configuration = config["evaluation"]["experiment_logger"]
logger = setup_evaluation_logger(
    model_name=experiment_name,
    labels=labels,
    run_dir=run_dir,
    **logger_configuration
)

## Evaluate Classification

In [ ]:
logger_configuration['classification_metrics']

In [ ]:
if config["evaluation"]["evaluate_classification"]:
    results = model.predict_with_epistemic_uncertainty(test_dataset, include_data=True)
    data, predictions_results, uncertainty_results = results
else:
    print("Skipped")

In [ ]:
if config["evaluation"]["evaluate_classification"]:
    logger.log_classification_metrics(data, predictions_results, uncertainty_results)

## Evaluate Aleatoric Model

In [ ]:
logger_configuration['aleatoric_uncertainty']

In [ ]:
if config["evaluation"]["evaluate_aleatoric_uncertainty"]:
    aleatoric_model = model.get_aleatoric_model()
    results_aleatoric = aleatoric_model.predict_with_aleatoric_uncertainty(test_dataset, include_data=True)
    aleatoric_data, aleatoric_predictions_results, aleatoric_uncertainty_results = results_aleatoric
else:
    print("Skipped")


In [ ]:
if config["evaluation"]["evaluate_aleatoric_uncertainty"]:
    logger.log_aleatoric_uncertainty(aleatoric_data, aleatoric_predictions_results, aleatoric_uncertainty_results)

## Evaluate Aggregation 

In [ ]:
logger_configuration['aggregation_metrics']

In [ ]:
if config["evaluation"]["evaluate_aggregation"]:
    aggregator_configuration = config["aggregation"]
    aggregator = setup_aggregator(**aggregator_configuration)
else:
    print("Skipped")

In [ ]:
if config["evaluation"]["evaluate_aggregation"]:
    if results is None:
        results = model.predict_with_epistemic_uncertainty(test_dataset, include_data=True)
    data, predictions_results, uncertainty_results = results
    agg = aggregator.predict_with_aggregation(test_dataset, predictions_results, uncertainty_results, include_data=True, verbose=1)
    aggregated_data, aggregated_predictions, aggregated_uncertainty = agg
else:
    print("Skipped")
gc.collect()

In [ ]:
if config["evaluation"]["evaluate_aggregation"]:
    logger.log_aggregation_metrics(test_dataset, data, predictions_results, uncertainty_results, 
                                   aggregated_data, aggregated_predictions, aggregated_uncertainty
        )

## WandB Sync

In [ ]:
wandb.finish()
#os.environ["WANDB_MODE"] = ""

In [ ]:
!wandb sync 

In [ ]:
reset_kernel()